In [79]:
from bs4 import BeautifulSoup
import requests
from github import Github
import os
from PIL import Image
from io import BytesIO
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from datetime import date, timedelta

ModuleNotFoundError: No module named 'github'

In [80]:
pip install github

  Using cached github-1.2.7-py3-none-any.whl (15 kB)
  Using cached aiohttp-3.8.1.tar.gz (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for aiohttp (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [107 lines of output]
      *********************
      * Accelerated build *
      *********************
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.1-arm64-cpython-311
      creating build/lib.macosx-11.1-arm64-cpython-311/aiohttp
      copying aiohttp/web_ws.py -> build/lib.macosx-11.1-arm64-cpython-311/aiohttp
      copying aiohttp/worker.py -> build/lib.macosx-11.1-arm64-cpython-311/aiohttp
      copying aiohttp/multipart.py -> build/lib.macosx-11.1-arm64-cpython-311/aiohttp
      copying aiohtt

### Import dataset

In [21]:
pd.set_option('display.max_colwidth', None)
df = pd.read_excel('/Users/wiesruyters/Documents/WhD/Repositories/NOS_scrape/Datasets/NOS_articles_221004-231004_corrected.xlsx')

In [29]:
article_df = df.copy()

In [30]:
article_df = article_df.drop('Unnamed: 0', axis=1)

##### Basic statistics

In [74]:
article_df['Month'] = article_df['Date'].dt.strftime('%B')
article_df['Weekday'] = article_df['Date'].dt.weekday

print(f'''
The columns present in the dataframe are respectively: \n{article_df.columns.to_list()}
\nThe shape of the dataframe is {article_df.shape}
\nThe top-10 categories entail:\n{article_df['Category'].value_counts().head(10)}
\nThe publishing rate (Oct '22 to Oct '23) per month was:\n{article_df['Month'].value_counts()}
''')


The columns present in the dataframe are respectively: 
['Article ID', 'Link', 'Title', 'Date', 'Time', 'Category', 'Images', 'Paragraphs', 'Month', 'Weekday']

The shape of the dataframe is (13387, 10)

The top-10 categories entail:
['Buitenland']                            3904
['Binnenland']                            2387
['Economie']                               908
['Politiek']                               688
['Binnenland', 'Buitenland']               456
['NH Nieuws', 'Regionaal nieuws']          277
['Cultuur & Media']                        274
['Omroep Brabant', 'Regionaal nieuws']     272
['Binnenland', 'Politiek']                 206
['Omroep West', 'Regionaal nieuws']        174
Name: Category, dtype: int64

The publishing rate (Oct '22 to Oct '23) per month was:
October      1224
March        1215
November     1181
December     1154
May          1130
June         1122
April        1089
January      1080
September    1073
July         1063
August       1029
February   

##### Correct datatypes

In [75]:
article_df['Date'] = pd.to_datetime(article_df['Date'], format='%Y-%m-%d').dt.date

### Build image scraper